In [1]:
from keras.applications.inception_v3 import InceptionV3
from keras.preprocessing import image
from keras.applications.inception_v3 import preprocess_input
from keras.models import Model
from keras.layers import Dense, GlobalAveragePooling2D
import numpy as np
import tensorflow as tf
import numpy as np
import os
import struct
import cv2

/Library/Frameworks/Python.framework/Versions/3.5/lib/python3.5/site-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
Using TensorFlow backend.
/Library/Frameworks/Python.framework/Versions/3.5/lib/python3.5/importlib/_bootstrap.py:222: RuntimeWarning: compiletime version 3.6 of module 'tensorflow.python.framework.fast_tensor_util' does not match runtime version 3.5
  return f(*args, **kwds)


In [2]:
# load in the MNIST data 
def read(dataset = "training", path = "."):
    """
    Python function for importing the MNIST data set.  It returns an tuple
    with the first element being the image and the second element
    being label for the given image.
    """

    if dataset is "training":
        fname_img = os.path.join(path, 'train-images.idx3-ubyte')
        fname_lbl = os.path.join(path, 'train-labels.idx1-ubyte')
    elif dataset is "testing":
        fname_img = os.path.join(path, 't10k-images-idx3-ubyte')
        fname_lbl = os.path.join(path, 't10k-labels-idx1-ubyte')
    else:
        raise (ValueError, "dataset must be 'testing' or 'training'")

    # Load everything in some numpy arrays
    with open(fname_lbl, 'rb') as flbl:
        magic, num = struct.unpack(">II", flbl.read(8))
        lbl = np.fromfile(flbl, dtype=np.int8)

    with open(fname_img, 'rb') as fimg:
        magic, num, rows, cols = struct.unpack(">IIII", fimg.read(16))
        img = np.fromfile(fimg, dtype=np.uint8).reshape(len(lbl), rows, cols)
    
    return img, lbl

mnist_images, mnist_labels = read()

In [3]:
mnist_images = np.expand_dims(mnist_images, axis = 3)
mnist_labels.shape

(60000,)

In [4]:
mnist_images.shape

(60000, 28, 28, 1)

In [5]:
# in order to fit the dataset as the network requires, make the depth = 3
# We simply divide the original value by 3 and concatenate the matrix

mnist_images_element = mnist_images/3
mnist_images_new = np.concatenate((mnist_images_element,mnist_images_element,mnist_images_element), axis = 3)
mnist_images_new.shape

(60000, 28, 28, 3)

In [6]:
# resize the data the match the network 
def resize_data(data):
    data_upscaled = np.zeros((data.shape[0], 140, 140, 3))
    for i, img in enumerate(data):
        large_img = cv2.resize(img, dsize=(140, 140), interpolation=cv2.INTER_CUBIC)
        data_upscaled[i] = large_img

    return data_upscaled

In [7]:
# training data preparation 
# shuffle the data in order to do Stochastic Gradient Descent
# In this case, MNIST has already been shuffled. 

n_classes = 10
n_samples = 2000

mnist_training_data = resize_data(mnist_images_new[:n_samples])
mnist_training_label = mnist_labels[:n_samples]
#random_index = np.random.permutation(n_data)
#amazon_training_data = amazon_data[random_index]
#amazon_training_labels = amazon_labels[random_index]
#print (amazon_training_data.shape, amazon_training_labels.shape)

In [8]:
# change labels to one_hot encoding 
mnist_training_label_one_hot = np.zeros((n_samples, n_classes))
# IMPORTANT: class 0 correspond to index 0, class 1 correspond to index 1
# class 0 represents digit 0
mnist_training_label_one_hot[np.arange(n_samples), mnist_training_label - 1] = 1
print (mnist_training_label_one_hot.shape)

(2000, 10)


In [9]:
# fine-tune the Inception V3 model
base_model = InceptionV3(weights = 'imagenet', include_top=False)

# add a global spatial average pooling layer
x = base_model.output 
x = GlobalAveragePooling2D()(x)

# add a fully-connected layer
x = Dense(1024, activation = 'relu')(x)

# add a logistic layer -- we have 31 classes
predictions = Dense(n_classes, activation='softmax')(x)

# this is the model we will train
model = Model(input = base_model.input, outputs = predictions)

# first: train only the top layers (which were randomly initialized)
# i.e. freeze all convolutional InceptionV3 layers
for layer in base_model.layers:
    layer.trainable = False

# compile the model (should be done *after* setting layers to non-trainable)
model.compile(optimizer='rmsprop', loss='categorical_crossentropy')

/Library/Frameworks/Python.framework/Versions/3.5/lib/python3.5/site-packages/ipykernel_launcher.py:15: UserWarning: Update your `Model` call to the Keras 2 API: `Model(inputs=Tensor("in..., outputs=Tensor("de...)`
  from ipykernel import kernelapp as app


In [10]:
# train the model on the new data for a few epochs
model.fit(x = mnist_training_data, y=mnist_training_label_one_hot, epochs = 2)

Epoch 1/2
2000/2000 [==============================] - 271s 135ms/step - loss: 2.5490
Epoch 2/2
2000/2000 [==============================] - 268s 134ms/step - loss: 1.2038


In [11]:
# after training the top layers, freeze the bottom N layers and train the remaining top layers 
# let's visualize layer names and layer indices to see how many layers
# we should freeze:
for i, layer in enumerate(base_model.layers):
   print(i, layer.name)

0 input_1
1 conv2d_1
2 batch_normalization_1
3 activation_1
4 conv2d_2
5 batch_normalization_2
6 activation_2
7 conv2d_3
8 batch_normalization_3
9 activation_3
10 max_pooling2d_1
11 conv2d_4
12 batch_normalization_4
13 activation_4
14 conv2d_5
15 batch_normalization_5
16 activation_5
17 max_pooling2d_2
18 conv2d_9
19 batch_normalization_9
20 activation_9
21 conv2d_7
22 conv2d_10
23 batch_normalization_7
24 batch_normalization_10
25 activation_7
26 activation_10
27 average_pooling2d_1
28 conv2d_6
29 conv2d_8
30 conv2d_11
31 conv2d_12
32 batch_normalization_6
33 batch_normalization_8
34 batch_normalization_11
35 batch_normalization_12
36 activation_6
37 activation_8
38 activation_11
39 activation_12
40 mixed0
41 conv2d_16
42 batch_normalization_16
43 activation_16
44 conv2d_14
45 conv2d_17
46 batch_normalization_14
47 batch_normalization_17
48 activation_14
49 activation_17
50 average_pooling2d_2
51 conv2d_13
52 conv2d_15
53 conv2d_18
54 conv2d_19
55 batch_normalization_13
56 batch_norma

In [12]:
# we chose to train the top 2 inception blocks, i.e. we will freeze
# the first 249 layers and unfreeze the rest:
for layer in model.layers[:249]:
   layer.trainable = False
for layer in model.layers[249:]:
   layer.trainable = True

# we need to recompile the model for these modifications to take effect
# we use SGD with a low learning rate
from keras.optimizers import SGD
model.compile(optimizer=SGD(lr=0.0001, momentum=0.9), loss='categorical_crossentropy')

In [13]:
model.fit(x = mnist_training_data, y=mnist_training_label_one_hot, epochs = 10)

Epoch 1/10
2000/2000 [==============================] - 346s 173ms/step - loss: 0.9046
Epoch 2/10
2000/2000 [==============================] - 345s 172ms/step - loss: 0.5112
Epoch 3/10
2000/2000 [==============================] - 298s 149ms/step - loss: 0.4272
Epoch 4/10
2000/2000 [==============================] - 307s 154ms/step - loss: 0.3959
Epoch 5/10
2000/2000 [==============================] - 334s 167ms/step - loss: 0.3837
Epoch 6/10
2000/2000 [==============================] - 263s 132ms/step - loss: 0.3350
Epoch 7/10
2000/2000 [==============================] - 299s 150ms/step - loss: 0.3077
Epoch 8/10
2000/2000 [==============================] - 341s 170ms/step - loss: 0.2880
Epoch 9/10
2000/2000 [==============================] - 336s 168ms/step - loss: 0.2586
Epoch 10/10
2000/2000 [==============================] - 339s 170ms/step - loss: 0.2647


In [15]:
# save weights to file
model.save('model.h5')
# Future usage: model.load_weights('.\\InceptionV3\\model.h5', by_name=False)